## Visualise fitting results 

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")

default_plot_setting()

In [ ]:
df_dds = CSV.read("../dt_surveys_master/master_dds.csv", DataFrame);

### Basic stats check

In [ ]:
println("Check max daly number of contacts")
tab = @pipe groupby(df_dds, [:key, :strat]) |> combine(_, :x => maximum => :max_cnt)
unstack(tab, :key, :strat, :max_cnt)

In [ ]:
for i in df_dds[:, :key] |> unique
    println(i)
end

In [ ]:
println("Check sample size")
tab = @pipe groupby(df_dds, [:key, :strat]) |> combine(_, :y => sum => :n_part)
unstack(tab, :key, :strat, :n_part)

In [ ]:
include("fit_utils.jl")
plot_zero_pct(df_dds)

# Create summary stats from fitting results

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")
#df_sum = DataFrame()
#keys = df_dds[:, :key] |> unique
#for k in keys
#    println(k)
#    df_sum = vcat(df_sum, create_summary_stat_one_data(k))
#end
#CSV.write("../dt_intermediate/fit_summary.csv", df_sum)
df_sum = CSV.read("../dt_intermediate/fit_summary.csv", DataFrame)
df_res = flag_minimum_IC(df_sum, :waic);

In [ ]:
df_obs = @pipe groupby(df_dds, [:key, :strat]) |>
    combine(_, :y => sum => :n_answer);

In [ ]:
include("fit_utils.jl")
df_obs_all = @subset(df_obs, :strat .== "all")
df_EVI = read_EVI_summary(df_obs);

### Mean comparison

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")

In [ ]:
obs_estimated_validation_plot(df_res)

# WAIC weights

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")
model_names = get_model_names()

In [ ]:
df_ana = prep_fmnl_vars(df_res)
sort!(df_ana, :n_sample)
ytk = df_ana[:, :key_pri];

In [ ]:
df_EVI_nhm = @subset(df_EVI, :strat .== "non-home");
plot_bar_waic(df_obs, df_res, ytk = ytk, df_EVI=df_EVI_nhm)

## Fractional multinomial regression for meta-regression

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")

In [ ]:
rem_lis = ["Leon_2013", "Leung_2017",  "CoMix2",
    #"CoMix2_at", "CoMix2_be", "CoMix2_dk",
    #"CoMix2_ee", "CoMix2_gr", "CoMix2_hr",
    #"CoMix2_it", "CoMix2_pl", "CoMix2_pt",
]
df_res_ana = @subset(df_res, @byrow (:key in rem_lis) == false);

In [ ]:
df_ana = prep_fmnl_vars(df_res_ana)
sort!(df_ana, :n_sample)
ytk = df_ana[:, :key_pri];
plot_bar_waic(df_obs, df_res_ana, ytk = ytk) #, df_EVI=df_EVI_nhm)

In [ ]:
df_ana = prep_fmnl_vars(df_res_ana);
#@select(df_ana, Not(model_names)) |> display
;

In [ ]:
pred, Y, x_names = one_hot_encoding_multi_vars(df_ana);

In [ ]:
include("main_utils.jl")
pred, Y, x_names = one_hot_encoding_multi_vars(df_ana);
@time chn = sample(model_fmnl(pred, Y), NUTS(), 2000; progress = true)

In [ ]:
#explore_chns(chn)

In [ ]:
include("main_utils.jl")
df_pred_cum = pred_fmnl_multi_vars(chn, pred)
plot_stacked_bar(df_pred_cum, model_names; xlabel="Prediced weight (%)")

In [ ]:
include("main_utils.jl")
# TODO: delete this part in the future
chn_res_multi = extract_chain_info(chn)
n_x = size(pred, 2)
forestplot_fmnl_multi_vars(chn_res_multi[Not([1, n_x+1]), :], x_names[2:end])

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")
chn_res = repeated_univariate_fmnl_reg(df_ana);

In [ ]:
include("fit_utils.jl")
plot_meta_reg(chn_res, chn_res_multi)

## Fitting validation

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")

In [ ]:
key = "Jonathan_2014"
plot_pdf_ccdf_validate(key)

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")
key = "Leung_2017_online"
plot_pdf_ccdf_validate(key)

In [ ]:
key = "Leung_2017_paper"
plot_pdf_ccdf_validate(key)

In [ ]:

pl = plot(; xlim = [0, 50], ylim = [-4, 0])
dd_home = res["dds"]["home"]


In [ ]:
df_tmp